<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:


      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-03 08:51:26
-------------------
qualified stocks: 92
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.17 L
Current:  1.39 C
-------------------
Today PnL: 84.98 K (0.61%)
Current PnL: -18.03 L (-12.13%)
CY Booked + Current PnL: -8.99 L (-6.05%)
-------------------
Total profit:  3.42 L
Total loss:  -21.46 L
-------------------
Total Booked + Current PnL: 17.99 L (14.78%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.49%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.70 L (57.26%)
Deployed:  1.22 C
Current:  1.39 C
CAGR/XIRR %: 8.5%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-16.60,63.0,H-LC,8.19,115352.0,11040.0,5606.0,0.19,10.58,4.86,15.96,9.0,1.97,0.84,29.27,X5K,ATH,METALS
26,DIXON,18940.15,-29.84,67.0,H-MC,6.64,140424.0,25488.0,11093.0,-0.16,22.18,7.90,31.83,81.0,2.30,1.02,45.55,X40N,ATH,IT
53,MEDANTA,1486.00,12.96,57.0,H-MC,15.39,161286.0,35086.0,11096.0,-0.40,27.80,6.88,36.59,108.0,3.16,1.18,43.85,XY24,NTT,HEALTHCARE
82,TTKPRESTIG,770.00,107.26,71.0,M-SC,7.31,89335.0,-11442.0,11533.0,1.14,-11.35,12.91,0.09,245.0,-0.99,0.65,17.37,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-15.41,55.0,H-LC,8.67,169004.0,12827.0,13267.0,0.14,8.21,7.85,16.71,22.0,0.97,1.23,27.08,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HAVELLS,2069.16,-0.37,62.0,H-MC,2.55,246917.0,-814.0,75878.0,0.55,-0.33,30.73,30.30,92.0,-0.01,1.80,12.92,X40,ATH,ELECTRICAL
38,ICICIGI,2260.25,-21.50,33.0,H-MC,4.65,134880.0,-1253.0,32385.0,-0.96,-0.92,24.01,22.86,91.0,-0.04,0.98,11.58,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RELIANCE,1533.00,-14.79,41.0,H-LC,7.85,213174.0,2808.0,25965.0,0.93,1.33,12.18,13.68,37.0,0.11,1.56,18.08,XY25,NTT,REFINERIES
47,JIOFIN,387.00,-1.31,41.0,H-LC,14.66,218225.0,819.0,52680.0,-0.73,0.38,24.14,24.61,48.0,0.02,1.59,55.40,XY24,BTT,FINANCE
33,HCLTECH,1943.91,0.65,42.0,H-LC,9.97,230115.0,-11809.0,75087.0,-0.18,-4.88,32.63,26.15,8.0,-0.16,1.68,8.74,X40,ATH,IT
1,ABB,7934.00,-40.84,46.0,H-LC,13.63,247114.0,-14505.0,133713.0,0.41,-5.54,54.11,45.57,7.0,-0.11,1.80,5.15,AR,NTT,ELECTRICAL
45,ITC,452.00,-38.34,47.0,H-LC,3.23,198901.0,-1237.0,22118.0,0.22,-0.62,11.12,10.44,4.0,-0.06,1.45,4.91,X40,NTT,FMCG


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.20,-4.91,57.0,H-LC,2.47,235958.0,-10193.0,89404.0,0.89,-4.14,37.89,32.18,16.0,-0.11,1.72,30.41,X200,ATH,IT
55,NESTLEIND,1377.00,-8.73,62.0,H-LC,2.88,281011.0,15585.0,41196.0,2.26,5.87,14.66,21.40,11.0,0.38,2.05,13.38,XY25,NTT,FMCG
45,ITC,452.00,-38.34,47.0,H-LC,3.23,198901.0,-1237.0,22118.0,0.22,-0.62,11.12,10.44,4.0,-0.06,1.45,4.91,X40,NTT,FMCG
20,CIPLA,1795.00,-19.44,55.0,H-LC,4.89,214980.0,10480.0,30936.0,-1.13,5.12,14.39,20.25,10.0,0.34,1.57,14.31,X40N,BTT,PHARMA
88,VBL,671.64,-11.61,49.0,H-LC,7.62,318490.0,2648.0,110006.0,0.78,0.84,34.54,35.67,5.0,0.02,2.32,14.81,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAMMAANCAP,326.00,-172.73,70.0,M-SC,7.78,83616.0,-18594.0,111987.0,11.11,-18.19,133.93,91.37,208.0,-0.17,0.61,35.74,XY25,NTT,FINANCE
54,NATIONALUM,247.44,-46.98,66.0,H-MC,0.99,106281.0,5777.0,24859.0,4.56,5.75,23.39,30.49,79.0,0.23,0.78,40.02,MH,ATH,METALS
58,RAJESHEXPO,518.00,1660.10,47.0,L-SC,8.77,48021.0,-89156.0,89252.0,4.08,-64.99,185.86,0.07,267.0,-1.00,0.35,19.43,OX40N,NTT,JEWELLERY
24,DABUR,735.00,-2.40,65.0,H-MC,3.76,212394.0,15366.0,74253.0,4.00,7.80,34.96,45.49,102.0,0.21,1.55,21.86,XY24,BTT,FMCG
12,BATAINDIA,2096.00,-35.72,54.0,M-SC,10.28,96064.0,-32606.0,77908.0,3.76,-25.34,81.10,35.20,219.0,-0.42,0.70,10.26,X40,NTT,FOOTWEAR


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,STARHEALTH,761.00,7.30,50.0,H-SC,2.73,159629.0,-40587.0,116609.0,-2.17,-20.27,73.05,37.97,144.0,-0.35,1.16,28.17,XY24,NTT,INSURANCE
40,INDIAMART,4850.92,-51.65,45.0,H-SC,12.52,135044.0,11708.0,122053.0,-2.01,9.49,90.38,108.45,119.0,0.10,0.99,33.97,AR,ATH,MISC
37,HONAUT,58357.33,-19.80,45.0,H-SC,3.47,112980.0,-14958.0,62094.0,-1.89,-11.69,54.96,36.84,143.0,-0.24,0.82,15.66,X40N,ATH,ELECTRICAL
72,SONACOMS,1006.00,-37.48,44.0,M-SC,13.30,77866.0,-23293.0,98181.0,-1.30,-23.03,126.09,74.03,202.0,-0.24,0.57,9.80,AR,BTT,AUTO
7,ATULAUTO,844.00,3175.68,44.0,M-SC,11.12,102164.0,-45204.0,97015.0,-1.29,-30.67,94.96,35.16,236.0,-0.47,0.75,4.35,XY24,NTT,AUTO


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TTKPRESTIG,770.0,107.26,71.0,M-SC,7.31,89335.0,-11442.0,11533.0,1.14,-11.35,12.91,0.09,245.0,-0.99,0.65,17.37,OX40N,NTT,DURABLES
83,UJJIVANSFB,60.0,84.36,45.0,H-SC,14.10,116289.0,-26190.0,45713.0,-0.55,-18.38,39.31,13.70,163.0,-0.57,0.85,37.25,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.34,53.0,H-SC,3.69,217791.0,-51876.0,88205.0,0.15,-19.24,40.50,13.47,138.0,-0.59,1.59,10.16,XY24,NTT,PAINTS
41,INDIGOPNTS,1408.0,112.05,46.0,M-SC,2.82,138967.0,-35592.0,35631.0,2.07,-20.39,25.64,0.02,221.0,-1.00,1.01,20.88,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,44.0,H-SC,3.15,113994.0,-29809.0,56552.0,0.73,-20.73,49.61,18.60,149.0,-0.53,0.83,25.80,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,4998.00,-26.76,50.0,H-LC,14.88,292481.0,-53475.0,177331.0,-0.04,-15.46,60.63,35.80,1.0,-0.30,2.13,3.61,X40,BTT,IT
43,INFY,2275.00,-18.16,50.0,H-LC,11.45,319223.0,6335.0,165358.0,-0.05,2.02,51.80,54.87,3.0,0.04,2.33,8.81,X40,BTT,IT
45,ITC,452.00,-38.34,47.0,H-LC,3.23,198901.0,-1237.0,22118.0,0.22,-0.62,11.12,10.44,4.0,-0.06,1.45,4.91,X40,NTT,FMCG
88,VBL,671.64,-11.61,49.0,H-LC,7.62,318490.0,2648.0,110006.0,0.78,0.84,34.54,35.67,5.0,0.02,2.32,14.81,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,46.0,H-LC,13.63,247114.0,-14505.0,133713.0,0.41,-5.54,54.11,45.57,7.0,-0.11,1.80,5.15,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,12.96,57.0,H-MC,15.39,161286.0,35086.0,11096.0,-0.40,27.80,6.88,36.59,108.0,3.16,1.18,43.85,XY24,NTT,HEALTHCARE
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,18.63,185920.0,35007.0,22292.0,1.30,23.20,11.99,37.97,93.0,1.57,1.36,54.37,AR,ATH,AUTO
26,DIXON,18940.15,-29.84,67.0,H-MC,6.64,140424.0,25488.0,11093.0,-0.16,22.18,7.90,31.83,81.0,2.30,1.02,45.55,X40N,ATH,IT
27,DMART,5201.00,-8.17,66.0,H-LC,13.89,229320.0,44839.0,25523.0,-1.16,24.31,11.13,38.14,38.0,1.76,1.67,37.49,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,12.96,57.0,H-MC,15.39,161286.0,35086.0,11096.0,-0.40,27.80,6.88,36.59,108.0,3.16,1.18,43.85,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,12.39,64.0,H-MC,3.42,194030.0,29360.0,38612.0,-0.41,17.83,19.90,41.28,89.0,0.76,1.42,27.24,X40N,ATH,AC
89,VOLTAS,1918.49,2.98,70.0,H-MC,5.34,216090.0,24348.0,71677.0,2.03,12.70,33.17,50.08,99.0,0.34,1.58,20.36,XY25,ATH,AC
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,18.63,185920.0,35007.0,22292.0,1.30,23.20,11.99,37.97,93.0,1.57,1.36,54.37,AR,ATH,AUTO
24,DABUR,735.00,-2.40,65.0,H-MC,3.76,212394.0,15366.0,74253.0,4.00,7.80,34.96,45.49,102.0,0.21,1.55,21.86,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,18.63,185920.0,35007.0,22292.0,1.30,23.20,11.99,37.97,93.0,1.57,1.36,54.37,AR,ATH,AUTO
54,NATIONALUM,247.44,-46.98,66.0,H-MC,0.99,106281.0,5777.0,24859.0,4.56,5.75,23.39,30.49,79.0,0.23,0.78,40.02,MH,ATH,METALS
90,WHIRLPOOL,2270.00,-40.79,56.0,M-SC,5.16,100028.0,8530.0,70220.0,1.29,9.32,70.20,86.07,223.0,0.12,0.73,45.26,XR,NTT,DURABLES
40,INDIAMART,4850.92,-51.65,45.0,H-SC,12.52,135044.0,11708.0,122053.0,-2.01,9.49,90.38,108.45,119.0,0.10,0.99,33.97,AR,ATH,MISC
91,WIPRO,420.00,-11.52,53.0,M-LC,7.15,155577.0,4632.0,104828.0,0.23,3.07,67.38,72.51,53.0,0.04,1.14,9.28,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAMMAANCAP,326.00,-172.73,70.0,M-SC,7.78,83616.0,-18594.0,111987.0,11.11,-18.19,133.93,91.37,208.0,-0.17,0.61,35.74,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-4.67,43.0,H-SC,16.38,88657.0,-12306.0,106832.0,1.28,-12.19,120.50,93.63,148.0,-0.12,0.65,32.05,SR,ATH,CHEMICALS
52,MASFIN,398.61,-19.15,47.0,H-SC,13.16,92085.0,-5895.0,27497.0,-1.00,-6.02,29.86,22.05,152.0,-0.21,0.67,33.85,XR,ATH,FINANCE
22,COFFEEDAY,80.00,-37.63,67.0,L-SC,31.25,91546.0,-22003.0,62059.0,0.97,-19.38,67.79,35.27,268.0,-0.35,0.67,124.06,XR,NTT,HOTELS
35,HINDALCO,756.01,-16.60,63.0,H-LC,8.19,115352.0,11040.0,5606.0,0.19,10.58,4.86,15.96,9.0,1.97,0.84,29.27,X5K,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,67.0,L-SC,31.25,91546.0,-22003.0,62059.0,0.97,-19.38,67.79,35.27,268.0,-0.35,0.67,124.06,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,18.63,185920.0,35007.0,22292.0,1.30,23.20,11.99,37.97,93.0,1.57,1.36,54.37,AR,ATH,AUTO
14,BLUESTARCO,2326.38,12.39,64.0,H-MC,3.42,194030.0,29360.0,38612.0,-0.41,17.83,19.90,41.28,89.0,0.76,1.42,27.24,X40N,ATH,AC
27,DMART,5201.00,-8.17,66.0,H-LC,13.89,229320.0,44839.0,25523.0,-1.16,24.31,11.13,38.14,38.0,1.76,1.67,37.49,X40N,NTT,FMCG
60,RELAXO,1176.00,-32.91,59.0,H-SC,7.18,62285.0,-53253.0,88245.0,-0.63,-46.09,141.68,30.28,136.0,-0.60,0.45,22.45,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.31
1,25,42.55
2,50,72.13


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    37.56
SC    36.39
MC    26.06
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.70
X40      15.20
X40N     13.93
XY25     11.30
AR        9.17
XR        8.82
OX40N     7.19
X5K       2.24
X200      1.72
MH        1.68
SR        1.06
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.57
H-MC    23.23
H-SC    21.76
M-SC    13.07
M-LC     6.96
M-MC     2.52
L-SC     1.56
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.00,0.72,31.61
IT,13.10,-14.91,71.91
FINANCE,7.14,-23.49,69.02
BANKS,7.13,-16.69,66.24
PAINTS,5.84,-15.34,41.16
MISC,5.75,-16.71,83.61
ELECTRICAL,5.58,-7.42,66.62
AUTO,4.49,-10.02,57.89
AC,3.71,4.62,27.68


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2843162.0,23
AR,1223687.0,10
XR,1087428.0,13
X40,883927.0,10
X40N,523672.0,11
OX40N,471341.0,10
XY25,432625.0,8
SR,198378.0,2
X5K,131460.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2794879.0,27
M-SC,1454282.0,17
H-LC,1415540.0,18
H-MC,1196634.0,18
M-LC,424673.0,5
M-MC,332786.0,2
L-SC,248055.0,3
L-MC,59782.0,1
L-LC,43612.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,945272.0,6
M-SC,XY24,861546.0,7
H-SC,AR,668049.0,5
H-LC,X40,590368.0,5
H-SC,XR,526863.0,6
H-MC,XY24,472545.0,5
H-LC,AR,372698.0,2
M-MC,XY24,332786.0,2
H-LC,X40N,220655.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
